### Введение

Получены данные в виде файла .csv с иформацией о стаже, образовании, количестве детей, семейном положении, доходах и целей кредита. Результатом исследования дожна стать информация для учета в построении модели кредитного скоринга. Данное исследование включает в себя изучение полученной информации, предварительная обработка пропусков и дкбликатов, а также лемматизация и категоризация данных. Результат работы будет в виде выводов с пояснениями ключевых показателей.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [4]:
import pandas as pd
from pymystem3 import Mystem
data = pd.read_csv('data.csv')
m = Mystem()

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [7]:
data.head(13)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Вывод

В датафрейме 21525 строк различного типа данных. 
Столбец days_employed имеет некорректные данные для общего трудового стажа в днях. 
В столбцах days_employed и total_income иимеются пропуски.

### Шаг 2. Предобработка данных

Для ответа на поставленные вопросы проверяем данные в колонке о детях

In [8]:
data['children'].value_counts()


 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Имеются аномальные данные о количестве детей: -1, 20(в большом отрыве)

In [9]:
print('Артефакты в данных о количестве детей составлют {:.1%}'.format((data.loc[data['children'] == 20]['children'].count()+data.loc[data['children'] == -1]['children'].count()) / data['children'].count())) 

Артефакты в данных о количестве детей составлют 0.6%


<div class="alert alert-success">
<h2> Комментарий ревьюера</h2>

Интересный способ посчитать доли в данных.
    
---
    
У метода `value_counts()` есть атрибут `normalize`, который принимает булевы значения. Этот атрибут позволяет посчитать доли в данных. 1
    
</div>

заполняем артефакты нулевым значением

In [10]:
data['children'].replace(-1 , 0, inplace=True)
data['children'].replace(20 , 0, inplace=True)
data['children'].value_counts()

0    14272
1     4818
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

Информация об образовании

In [11]:
data['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

Информация о семейном положении

In [12]:
data['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

### Обработка пропусков

Производим проверку пропусков функцией isna() с суммированием пропусков

In [13]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [14]:
data_none = data.loc[data['total_income'].isna()]
data_none.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


Имеются 2174 отсутствующих значения в двух столбцах. Посмотрим на данные трудового стажа в 'days_employed'

Колонка days_employed имеет отрицательные значения с пропусками. Вероятно это ошибка выгрузки. 

In [15]:
#print(data.head())
data['days_employed'].describe()

count     19351.000000
mean      63046.497661
std      140827.311974
min      -18388.949901
25%       -2747.423625
50%       -1203.369529
75%        -291.095954
max      401755.400475
Name: days_employed, dtype: float64

Отрицательные значения по модулю могут быть настоящим стажем. Имеются также аномальные значения стажа в днях более 100 лет. Смотрим статистические показатели, изучим аномальные данные.

In [16]:
#print(data.sort_values(by= 'days_employed', ascending = False))
print(data.loc[data['days_employed'] > 0 ]['total_income'].describe())

count      3445.000000
mean     137124.105624
std       80242.210917
min       20667.263793
25%       82876.335652
50%      118514.486412
75%      169746.263276
max      735103.270167
Name: total_income, dtype: float64


Медиана 118514. Даже если принять стаж в часах, то ~ 60 лет (118514/8/247) (предположительное количество часов/рабочие часы/количество рабочих дней в году) стаж не является правдоподобным.

Для замены  среднее (mean) не годится. Имеются сильно кореллирующие значения. Можно заполнить медианным значением отрицательных показателей.

In [17]:
median_days_employed = data[data['days_employed'] < 0]['days_employed'].median()
print('Медиана отрицательных значений', median_days_employed)

Медиана отрицательных значений -1630.0193809778216


Заменяем пропуски на медиану отрицательных значений. 

In [18]:
data['days_employed'].fillna(median_days_employed, inplace=True)
data.isna().sum()

children               0
days_employed          0
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [19]:
#data.loc[data['days_employed'] > 0 ] = data.loc[data['days_employed'] < 0 ]['total_income'].median()
#data['days_employed'].describe()
#print(data.loc[data['days_employed'] > 0 ]['total_income'].describe())

<div class="alert alert-warning">
<h2> Комментарий ревьюера</h2>

Как ты думаешь с чем связано наличие отрицательных значений ? 
    
Рекомендую не избавляться от используемых данных.
    
</div>

Колонка total_income:

In [20]:
pass_data = data['total_income'].isna().sum() / data['total_income'].count()
print('Пропусков в данных о доходах:', data['total_income'].isna().sum())
print('Пропуски составляют:{:.0%}'.format(pass_data))
print('Максимальный доход:', data['total_income'].max())
print('Минимальный доход :', data['total_income'].min())
print('Средний доход :', data['total_income'].mean())
median_income = data['total_income'].median()
print('Медиана ежемесячных доходов:', median_income)
#print(data['total_income'].describe()) 

Пропусков в данных о доходах: 2174
Пропуски составляют:11%
Максимальный доход: 2265604.028722744
Минимальный доход : 20667.26379327158
Средний доход : 167422.30220817294
Медиана ежемесячных доходов: 145017.93753253992


<div class="alert alert-success">
<h2> Комментарий ревьюера</h2>

Хорошо, что изучил стандартные статистические показатели.
    
---
    
У нас есть готовый метод `describe()`, который позволяет одним разом изучить все эти параметры.
    
</div>

Остались пропуски в солбце 'total_income'. Для снижения вероятности разброса данных заменяем пропуски в доходах медианным значением ежемесячного дохода по категориям образования и занятости. Среднее чувствительно к экстремальным значениям. Результат запишем в столбец 'income_median_by_edu_income'

In [21]:
#data.groupby('education').aggregate('median')
def m_total_inc(column): # рассчитывет медиану  ежемесячного дохода у каждой категории столбца column
    return data.groupby(column)['total_income'].transform(lambda x : x.median())

data['income_median_by_edu_income'] = (m_total_inc('education') + m_total_inc('income_type')) / 2 
#data.head()

In [22]:
data['total_income'].fillna(data['income_median_by_edu_income'], inplace=True)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,income_median_by_edu_income
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,159109.854612
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,139618.871659
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,138850.122370
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,139618.871659
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,127578.916441


Проверяем на пропуски повторно

In [23]:
print('Пропусков данных в ежемесячных доходах:', data['total_income'].isna().sum())

Пропусков данных в ежемесячных доходах: 0


### Вывод

Обнаружены пропущеные значения в двух столбцах: общий стаж и общий доход, в одних и тех-же строках. Вероятнее всего это ошибка выгрузки. В колонке трудового стажа данные не соответствуют реальным данным: Много отрицательных значений или значений соответствующим 100 лет и более. Пропуски были обработаны следующим образом: В информации о стаже пропуски заполнены медианным отрицательным значением, т.к. положительные значения являются аномальными для эток категории. В поле ежемесячного дохода отсутствующие значения были заменены медианой изза имеющихся экстремальных значений в верхнем диапазоне доходов от срезов по образованию и типу занятости для снижения вероятности разброса в данных. 

<div class="alert alert-success">
<h2> Комментарий ревьюера</h2>

Очень хорошо, что обратываем пропуски с помощью медианных значений и понимаем почему именно медианой. 
       
Давай попробуем сделать замену с помощью обработки по пропусков по группам то есть у нас есть информация по профессия, образованию. Поможет нам в этом методы `groupby()`, `trasform()` и `fillna()`.
    
Руководствоваться будем следующей логикой:
    
1. Создадим отдельный столбец, в который поместим медианное значение для каждого объекта исходя из группы профессий. В этом нам поможет метод `transform()`
2. Далее заменим пропуски используя полученный столбец
   
---
    
Таким образом мы сможем снизить вероятность сильного разброса в данных.
    
</div>

### Замена типа данных

Для лучшего восприятия заменим вещественный тип столбца 'total_income' на числовой

In [24]:
data['total_income'] = data['total_income'].astype('int')
#data.head()

In [25]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   children                     21525 non-null  int64  
 1   days_employed                21525 non-null  float64
 2   dob_years                    21525 non-null  int64  
 3   education                    21525 non-null  object 
 4   education_id                 21525 non-null  int64  
 5   family_status                21525 non-null  object 
 6   family_status_id             21525 non-null  int64  
 7   gender                       21525 non-null  object 
 8   income_type                  21525 non-null  object 
 9   debt                         21525 non-null  int64  
 10  total_income                 21525 non-null  int32  
 11  purpose                      21525 non-null  object 
 12  income_median_by_edu_income  21525 non-null  float64
dtypes: float64(2), i

### Вывод

Для замены типа данных использовался метод astype().  


<div class="alert alert-success">
<h2> Комментарий ревьюера</h2>

Как думаешь, для чего мы делаем замену типов в данных  ?    
</div>

<div class="alert alert-success">
Если честно, не знаю. ни мне ни компьютеру float не мешает. Возможно, вот, в задании есть "заменить тип данных", а больше менять и нечего :) Но хотелось бы узнать 

### Обработка дубликатов

Посмотрим на дубликаты в таблице целиком методом duplicated(). просуммируем результат

In [26]:
print('Обнаружено дубликатов строк:', data.duplicated().sum())

Обнаружено дубликатов строк: 54


Удаляем дубликаты методом drop_duplicates() с параметром inplace=True для фактического удаления повторяющихся строк

In [27]:
data.drop_duplicates(inplace=True)

Проверяем дубликаты повторно

In [28]:
print('Обнаружено дубликатов строк:', data.duplicated().sum())

Обнаружено дубликатов строк: 0


Проверяем категориальные данные на дубликаты, приводим в порядок столбец 'education'

In [29]:
print(data['education'].value_counts())

среднее                13705
высшее                  4710
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   273
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64


Значения находятся в различном регистре. Приводим к нижнему регистру методом  str.lower(). Проверяем результат.

In [30]:
data['education'] = data['education'].str.lower()
print(data['education'].value_counts())

среднее                15188
высшее                  5251
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64


### Вывод

Обнаружены дубликаты строк полностью, которые могли появиться по техническим причинам. Также Обработаны дубликаты с одним значением, с разным регистром, природа появления которых, вероятно на этапе сбора данных.

### Лемматизация

Для ответа на вопросы имеется необходимость лемматизировать колонку 'purpose' о целях получения кредита.

In [31]:
print(data['purpose'].value_counts())

свадьба                                   793
на проведение свадьбы                     773
сыграть свадьбу                           769
операции с недвижимостью                  675
покупка коммерческой недвижимости         662
операции с жильем                         652
покупка жилья для сдачи                   652
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          625
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

Изучив данные о целях получения кредита, значения можно разделить на 4 категории: 'свадьба' 'недвижимость' 'автомобиль' 'образование'. Добавим столбец лемм 'purpose_lemm' с категориями в таблицу с помощью функции, которая разберет списки лемм, объединенных методом join() от каждой строки на категории 'свадьба', 'недвижимость', 'автомобиль', 'образование'. Леммы "жильё" также отнесутся к 'недвижимость'.

In [ ]:
def lemm_purpose(row):
    lemmas = ' '.join(m.lemmatize(row))
    if 'свадьба' in lemmas:
        return 'свадьба'
    elif ('жилье' or 'недвижимость') in lemmas:        
        return 'недвижимость'    
    elif 'автомобиль' in lemmas:    
        return 'автомобиль'   
    elif 'образование' in lemmas:    
        return 'образование'
        
data['purpose_lemm'] = data['purpose'].apply(lemm_purpose)


Результат лемматизации в колонке 'purpose_lemm':

In [ ]:
print(data['purpose_lemm'].value_counts())

In [ ]:
data.head()

### Вывод

Приведение строк к лемме проведено с помощью библиотеки pymystem3. mystem  это морфологический анализатор русского языка с поддержкой снятия морфологической неоднозначности. Т.е. все слова в строках приводятся в именительный падеж, единственное число, мужской род, а глаголы, причастия и деепричастия в инфинитиве несовершенного рода. Pymystem3 по умолчанию выдает список лемматизированных слов, который с помощью функции lemm_purpose(row) категоризируем и записывем в новое поле.

### Категоризация данных

Определяем функцию сводной таблицы с добавлением столбца отношения должников к общему количству заемщиков. Параметры функции: Строки - 'index', колонки - принимаемые данные из 'column'. Функция возвращает фрейм данных с колонками из значений column и сторками index, сортированный по отношению должников в порядке возрастания.

In [ ]:
def t_pivot(index, column): # Функция сводной таблицы. Строки - index, значение полей - column
    t = data.pivot_table(index=[index], values='total_income', columns= column ,aggfunc='count') #Сводная таблица
    t['ratio'] = (t[1] / (t[0] + t[1])) # добовляется столбец отношение 
    return t.sort_values(by = 'ratio', ascending = True) #Сортировка по отношению


<div class="alert alert-success">
<h2> Комментарий ревьюера</h2>

Здорово, что сделал функцию для расчетов! :)    
</div>

#### Долг и дети

Создаем таблицу 'children_debt' с данными о детях и задолженностях.

In [ ]:
children_debt = data.pivot_table('debt', index='children', aggfunc='sum') #Сводная таблица с суммой невозвратов 
children_debt['count_take'] = data['children'].value_counts() #Количество займов Всего
children_debt['non_return'] = children_debt['debt'] / children_debt['count_take']#Доля невозвратов
#children_debt

#### Долг и семейное положение

Создаем сводную таблицу: Индекс - 'family_status', колонки - данные из 'debt' .
вычисляем отношение должников к общему количеству

In [ ]:
#family_debt_pivot = data.pivot_table(index=['family_status'], values='family_status_id', columns= 'debt' ,aggfunc='count')
#family_debt_pivot['ratio'] = (family_debt_pivot[1] / (family_debt_pivot[0] + family_debt_pivot[1]))
#family_debt_pivot.head()
family_debt_pivot = t_pivot('family_status', 'debt')
print(family_debt_pivot)

#### Долг и доходы

Создадим категории дохода в таблице data в столбце 'income_level' по данным поля 'total_income': 
менее 90000 низкий доход;
от 90000 до 160000 - средний доход;
от 160000 до 250000 - выше среднего;
более 250000 - высокий доход

In [ ]:
def income_level(row):
    
    income = row['total_income'] 
    print(income)
    if income >= 250000:
        return 'Высокий доход'
    elif 250000 > income >= 160000:
        return 'Выше среднего'
    elif 160000 > income >= 90000:
        return 'Средний доход'
    elif 90000 > income:
        return 'Низкий доход'
    
data['income_level'] = data.apply(income_level, axis=1)
#data['income_level'].value_counts()
#data.head()


<div class="alert alert-success">
<h2> Комментарий ревьюера</h2>

Расскажи пожалуйста каким образом получились данные уровни доходов ? 
Сначала была идея от медианы половины делить пополам на несколько уровней. алгоритм по типу обхода графов. Потом подумал мало, надо график, применить фильтры, исследовать функцию найти локальные минимумы максимумы и ими пользоваться. Но время... поменял несколько значений для более показательного результата. И вот как есть.
    Надеюсь можно сюда писать )
</div>

<div class="alert alert-success">
<h2> Комментарий ревьюера</h2>

Лучше писать свои комментарии в отдельной ячейке, с другим цветом. Например с классом **info**, вместо **success**.
    
Посмотри в открытом доступе квантили/квартили/перцентили (это всё одно и то же). С помощью станадратных статистических действий, сможем быстро определить нужные нам уровни доходов.
</div>

Создаем сводную таблицу: 

In [ ]:
#income_pivot = data.pivot_table(index=['income_level'], values='total_income', columns= 'debt' ,aggfunc='count')
#income_pivot['ratio'] = (income_pivot[1] / (income_pivot[0] + income_pivot[1]))
income_pivot = t_pivot('income_level', 'debt')
print(income_pivot)

#### Долг и цели

Создадим сводную таблицу по лемметизированным целям кредита, добавим долю задолженности в последнем столбце

In [ ]:
#purpose_pivot = data.pivot_table(index=['purpose_lemm'], values='total_income', columns= 'debt' ,aggfunc='count')
#purpose_pivot['ratio'] = (purpose_pivot[1] / (purpose_pivot[0] + purpose_pivot[1]))
purpose_pivot = t_pivot('purpose_lemm', 'debt')
print(purpose_pivot)

### Вывод

Данные подготовлены для ответов на поставленные вопросы. По каждой категории составлены таблицы для удобного отображения.
Объединенины избранные данные в произвольныx группах по заданному критерию.

<div class="alert alert-success">
<h2> Комментарий ревьюера</h2>

Обрати пожалуйста внимание, что мы выполняем повторяющиеся действия. Это означает — оптимизация.

То есть мы можем сделать например функцию, которая будет выполнять все наши расчеты и выводить необходимые нам результаты.

</div>

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [ ]:
childfree = children_debt.loc[0, 'non_return']
print('Невозврат кредитов в срок среди бездетных {:.2%}'.format(childfree))
inchild = children_debt.loc[1:, 'debt'].sum() / children_debt.loc[1:, 'count_take'].sum()
print('Невозврат кредитов в срок среди родителей {:.2%}'.format(inchild))

### Вывод

Невозврат кредитов в срок среди заемщикоов, имеющих детей выше.


<div class="alert alert-success">
<h2> Комментарий ревьюера</h2>

Верно.    
</div>

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [ ]:
family_debt_pivot

### Вывод

Имеется зависимость невозврата. Те, кто был в браке более ответственно относятся к возврату кредита


<div class="alert alert-success">
<h2> Комментарий ревьюера</h2>

Верно.    
</div>

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [ ]:
income_pivot

### Вывод

Имеется зависимость. Заемщики с высоким и низким доходом отдают кредиты лучше


<div class="alert alert-success">
<h2> Комментарий ревьюера</h2>

Верно.    
</div>

- Как разные цели кредита влияют на его возврат в срок?

In [ ]:
purpose_pivot

### Вывод

Наиболее надежные кредиты - кредиты на недвижимость


<div class="alert alert-success">
<h2> Комментарий ревьюера</h2>

Верно.    
</div>

### Шаг 4. Общий вывод

  От кредитного отдела банка была получена статистика платежеспособности клиентов. Задача - выявить зависимости от семейного положения, наличия детей, дохода и целей кредита на факт погашения кредита в срок. Данные получены в виде файла .csv.
    В файле 21525 записей с иформацией о стаже, образовании, количестве детей, семейном положении, доходах и целей кредита. В данных были обнаружены и устранены пропуски значений путем замены значения медианой от категорий образования и типа занятости, дубликаты строк и значений разного регистра, некорректные значения. Была проведена лемматизация и категоризация целей кредита.
    По данным таблицы после предобработки были составлены сводные таблицы с выводом ключевых показателей на каждый из поставленных вопросов.
    В проекте получены следующие результаты:
1 Бездетные отдают кредиты лучше:невозврат   7.53%, против 9.24% у имеющих детей
2 Заемщики с жизненным опытом более надежные невозврат: Холостые- 9.7%, В гражданском браке- 9.3%, в браке - 7.5%, разведенные- 7.1%, вдовцы- 6.5%
3 Низкий и высокий уровни дохода способствуют возврату кредита в срок - 7.8% и 6.8% соответственно. средние показатели дохода ~ 8%
4 Недвижимость является самым надежным кредитом - невозврат 6.9%. Свадьба - 7.9%, образование и автомобиль ~9% 
    

<div class="alert alert-danger">
<h2> Комментарий ревьюера</h2>

Финальный вывод и есть главный результат твоей работы. Стоит писать его подробно по результатам проведенной работы. В нем можно приводить полученные в ходе работы значения. Также можно расписать все, что было сделано в работе.
    
Самое главное — это вывод должен содержать в себе ответ на поставленную задачу.

</div>

<div class="alert alert-success">
<h2> Комментарий ревьюера</h2>

Выводы понятны и логичны, а самое главное подкреплены выявленными фактами.
    
В целом сам проект выполнен на хорошем уровне! На протяжении всей работы чувствуется глубина проработки задачи и это безусловный плюс для нас. Было использовано большое количество методов, которые будут помогать тебе в дальнейших проектах.
    
Не стоит забывать про оформление своей работы, это важный такой же важный этап. Рекомендую «не стесняться» использовать ячейки типа **markdown** там где нам необходимо отобразить свои размышления, в том числе с использованием дополнительной стилизации.
  
Успехов тебе в новых работах! 
    
</div>

<div class="alert alert-success">
<h2> Комментарий ревьюера</h2>

Рекомендую посмотреть на дополнительную стилизацию ячеек типа **markdown**.
    
**Жирный**
*Курсив*
    
--- 

# Заголовок первого уровня

--- 
    
## Заголовок первого уровня
  
--- 
Списки:
    
- один 
- два
- три
   
---
    
1. Раз
2. Раз раз
3. Раз два три
    
--- 
    
Отображение `переменных`
    
    
</div>